# 1.1. Flask 와 Database

- 실제 웹 서비스에서는 다양한 DB 를 사용
<br>
<br>
- DB 변경이 적은 경우, **MySQL** 과 같은 **RDBMS** 를 사용
    - 정보가 많고, 보안 및 안정성이 뛰어남
<br>
<br>
- DB 변경이 잦은 경우, **NoSQL** 을 사용
<br>
<br>
- 추가적으로, 웹 서비스의 성능 향상을 위해 **redis** 와 같은 **in-memory** 방식의 DB 도 많이 사용

### **`MySQL vs MongoDB`**

- 반면, 중요도가 높고 안정성이 필요한 데이터를 처리할 때에는 **MySQL** 이 적합
- 방대하지만, 일부 유실되도 큰 문제가 발생하지 않는 데이터를 처리할 때는 **MongoDB** 가 적합

# 1.2. Flask 와 MySQL

### **`SQLAlchemy vs Pymysql`**

- Flask 에서는 Django 와 같이 내장 **ORM** 을 제공하지 않으므로, 별도의 **third-party lib** 이 필요
- 이 때, **SQLAlchemy** 를 주로 사용
<br>
<br>
- 하지만, SQL 이외로 별도의 **SQLAlchemy** CRUD 문법 및 기법을 익혀야 함
- DB 를 객체로 접근하는 방식이 직접 DB 에 접근하는 방식에 비해 성능 상 이점이 없음

### **`Install MySQL on Local Mac`**

```
brew install mysql
mysql.server start
mysql -u root
```

### **`Access MySQL with Setting`**

```
GRANT ALL PRIVILEGES ON *.* TO 'root'@'localhost';
CREATE USER 'aiden'@'localhost' IDENTIFIED BY 'password';
GRANT ALL PRIVILEGES ON *.* TO 'aiden'@'localhost';
flush privileges;
```

- `root` 계정으로 자신의 PC 내 MySQL 서버에 접근할 경우, 모든 권한 부여
- `aiden` 계정의 `password` 를 통해 사용자 생성
- `aiden` 계정으로 자신의 PC 내 MySQL 서버에 접근할 경우, 모든 권한 부여
- DB 에 영구적으로 모든 변경 내역 즉시 반영

### **`Pymysql`**

- **MySQL** 을 **Python** 에서 사용할 수 있게 하는 라이브러리

```
pip install PyMySQL
```

```
import pymysql
pymysql 모듈의 connect()                                   --> host, port, login, password, db 지정 (MySQL 에 연결)
Connection 객체의 cursor()                                 --> Cursor 객체
Cursor 객체의 execute()                                    --> SQL 문 DB 에 전송
Cursor 객체의 fetchall(), fetchone(), fetchmany()          --> SQL Query 후 데이터 처리
Connection 객체의 commit()                                 --> DML(INSERT, UPDATE, DELETE) 실행 후 DB 에 반영
Connection 객체의 close()                                  --> DB 연결 종료
```

### **`Access MySQL with Pymysql`**

- **`pymysql.connect()`**
    - **`host`**: 접속할 MySQL Server 주소
    - **`port`**: 접속할 MySQL Server 주소의 Port 번호
    - **`user`**: MySQL ID
    - **`passwd`**: MySQL ID 의 password
    - **`db`**: 접속할 Database
    - **`charset='utf8'`**: MySQL 에서 데이터 가져올 때 한글이 깨질 우려가 있으므로 설정에 추가

```
mysql -u root
CREATE DATABASE blog_db;
```

- 아래 예시는 **AWS RDS** 를 사용한 예시입니다.

In [24]:
host = "blog-db-test.c5jg0g5uf2zu.ap-northeast-2.rds.amazonaws.com"

In [25]:
import pymysql

db_conn = pymysql.connect(
        host=host,
        port=3306,
        user='root',
        passwd='password',
        db='blog_db',
        charset='utf8')

rds_db = db_conn.cursor()
rds_db

### **`테이블 수 확인`**

In [26]:
sql = 'SHOW TABLES;'
rds_db.execute(sql)

1

### **`테이블 생성`**

In [14]:
sql = '''CREATE TABLE user_info (
    USER_ID INT UNSIGNED NOT NULL AUTO_INCREMENT, 
    USER_EMAIL VARCHAR(100) NOT NULL, 
    BLOG_ID CHAR(4), 
    PRIMARY KEY(USER_ID)
    );'
'''

rds_db.execute(sql)
db_conn.commit()

- 데이터를 변경한 후에는 위와 같이 commit() 을 해주어야 한다.

### **`데이터 추가`**

In [16]:
user_email = 'test@test.com'
blog_id = 'A'

In [20]:
sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (str(user_email), str(blog_id))
rds_db.execute(sql)
db_conn.commit()

### **`데이터 조회`**

- **fetch Methods**
    - **fetchall()**: Fetch all the rows
    - **fetchmany(size=None)**: Fetch several rows
    - **fetchone()**: Fetch the next row

In [21]:
sql = "SELECT * FROM user_info"

rds_db.execute(sql)
results = rds_db.fetchall()
for result in results:
    print (result, type(result))

(1, 'test@test.com', 'A') <class 'tuple'>


In [22]:
sql = "SELECT * FROM user_info WHERE USER_EMAIL = '" + str('test@test.com') + "'"

rds_db.execute(sql)
results = rds_db.fetchall()
for result in results:
    print (result, type(result))

(1, 'test@test.com', 'A') <class 'tuple'>


### **`DB 연결 해제`**

In [23]:
db_conn.close()

# 1.3. Flask 와 MongoDB

### **`Install MongoDB on Local Mac`**

- **mongodb** 설치 및 실행
<br>

```
xcode-select --install
brew tap mongodb/brew
brew install mongodb-community@5.0

brew services start mongodb-community@5.0
```

- **mongodb** 가 실행되어 있지만 **재실행**하고 싶은 경우
<br>

```
brew services restart mongodb-community@5.0
```

- **mongodb** 를 **중지**하고 싶은 경우
<br>

```
brew services stop mongodb-community@5.0
```

### **`Access MongoDB`**

In [31]:
import pymongo

username = ''
password = ''
ip_address = 'localhost'
connection = pymongo.MongoClient()
connection = pymongo.MongoClient('mongodb://%s' % (ip_address))

# connection = pymongo.MongoClient('mongodb://%s:%s@%s' % (username, password, ip_address))
blog_session_db = connection.blog_session_db
blog_ab = blog_session_db.blog_ab

### **`Mongodb 연결 확인`**

- **mongodb** 연결을 한번 해놓으면 이론적으로는 해당 객체를 사용하므로 **mongodb** CRUD 를 실행하면 됨
<br>
<br>
- 하지만, 실제로는 **mongodb** 가 다양한 원인으로 다운되거나, 연결이 해제되는 경우가 있음
  - 이 경우, 연결이 되어있음을 가정하고, CRUD를 실행할 경우 에러가 남
<br>
<br>
- 따라서 연결된 객체가 아직 **mongodb** 에 연결이 되어있는지 체크하는 방법이 필요
  - 물론, 이 방법도 **mongodb** 가 아예 다운된 경우등을 체크하도록 코드를 더 정교하게 작성할 수도 있지만,
  - 문제를 **mongodb** 는 다운되더라도 재기동된다고 가정하고, (docker 등 다른 시스템을 통해 해당 기능 활성화)
  - 연결이 해제된 경우만 체크하는 기법을 사용

In [32]:
connection.admin.command('ismaster')

{'ismaster': True,
 'topologyVersion': {'processId': ObjectId('626ab93f3c66b7e293ed459b'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2022, 4, 28, 16, 12, 6, 498000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 11,
 'minWireVersion': 0,
 'maxWireVersion': 13,
 'readOnly': False,
 'ok': 1.0}

In [33]:
connection.server_info()

{'version': '5.0.7',
 'gitVersion': 'b977129dc70eed766cbee7e412d901ee213acbda',
 'modules': [],
 'allocator': 'system',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [5, 0, 7, 0],
 'openssl': {'running': 'Apple Secure Transport'},
 'buildEnvironment': {'distmod': '',
  'distarch': 'x86_64',
  'cc': '/Applications/Xcode10.2.0.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: Apple LLVM version 10.0.1 (clang-1001.0.46.3)',
  'ccflags': '-Wno-error=unknown-warning-option -isysroot /Applications/Xcode10.2.0.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk -mmacosx-version-min=10.14 -target darwin18.0.0 -arch x86_64 -Werror -include mongo/platform/basic.h -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautologica

### **`INSERT`**

In [34]:
blog_ab.insert_one({'emailid':'test@test.com'})

### **`SELECT`**

In [35]:
blog_ab.find_one({'emailid':'test@test.com'})

{'_id': ObjectId('626abd00625a312705229a20'), 'emailid': 'test@test.com'}

### **`DELETE`**

In [36]:
blog_ab.delete_one({'emailid':'test@test.com'})

### **`SELECT *`**

In [37]:
blog_logs = blog_ab.find()
for log in blog_logs:
    print(log)